In [ ]:
from google.colab import drive

MOUNTPOINT = '/content/gdrive'
drive.mount(MOUNTPOINT)

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

input_dataset = '/content/gdrive/My Drive/Colab Notebooks/datasets/'
output_dataset = '/content/gdrive/My Drive/Colab Notebooks/datasets/ISIC_2019_Training_GroundTruth.csv'

train_x = tf.keras.utils.image_dataset_from_directory(directory=input_dataset, labels=None, shuffle=False, image_size=(256,256))

# 15% test and val split with 70% train
val_x = train_x.take(3799)
train_x = train_x.skip(3799)
test_x = train_x.take(3799)
train_x = train_x.skip(3799)

features = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']


df = pd.read_csv(output_dataset)
train_y = tf.data.Dataset.from_tensor_slices(dict(df))

# 15% test and val split with 70% train
val_y = train_y.take(3799)
train_y = train_y.skip(3799)
test_y = train_y.take(3799)
train_y = train_y.skip(3799)


#val_y, test_y, train_y = np.split(df.sample(frac=1), [int(.15*len(df)), int(.30*len(df))])



#train_y = tf.data.Dataset.from_tensor_slices(tf.cast(train_y[features].values, tf.int32))


# need to convert y data to tf.dataset and then zip them together
train_ds = tf.data.Dataset.zip((train_x , train_y))
val_ds = tf.data.Dataset.zip((val_x , val_y))
test_ds = tf.data.Dataset.zip((test_x , test_y))


In [ ]:
height = 256
width = 256
channels = 3

model = model(height, width, channels)
history =  model.fit(train_ds, validation_data=val_ds, batch_size = 16, epochs=10)

#but what about this ds????
predict_test = model.predict(test_x)